In [1]:
import urllib
import requests
from IPython.core.display import HTML
def css_styling():
    styles = requests.get('https://raw.githubusercontent.com/saadtony/NumericalMethods/master/styles/custom.css')
    return HTML(styles.text)
css_styling()

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.animation as animation
%matplotlib inline
%config InlineBackend.figure_format='svg'
plt.rcParams['animation.html'] = 'html5'

# Problem 1
## Derive the modified equation for an FTCS scheme for the linear advection equation:

$$u_t = -cu_x $$

The numerical form of a this partial differential equation actually solves an equation of the form:

$$u_t = a_1u_x + a_2u_{xx} + a_3u_{xxx} + a_4u_{xxxx} + ... $$

where odd-ordered derivatives represent advection and dispersion terms, and even-ordered derivatives represent diffusion terms.

Re-writing the PDE in numerical form following an FTCS scheme:

$$\frac{u^{n+1}_{i}-u^{n}_{i}}{\Delta t} \  +\  c\frac{u^{n}_{i+1}-u^{n}_{i-1}}{\Delta x} =0$$

Expanding the $u^{n+1}_i$ term as a Taylor series about time $n$:

$$u^{n+1}_{i}=u^{n}_{i}+\frac{\partial u^{n}_{i}}{\partial t} \Delta t+\frac{\partial^{2} u^{n}_{i}}{\partial t^{2}} \Delta t+...$$

which can be written in shorthand as:

$$u^{n+1}_{i}=u^{n}_{i}+\Delta tu^{n}_{it}+\frac{\Delta t^{2}}{2} u^{n}_{itt}+\frac{\Delta t^{3}}{3!} u^{n}_{ittt}+...$$

A brief note about this notation: The first index in the subscript indicates the velocity at point $i$, and is not a derivative. Similarly, we can expand the $u^{n}_{i\pm 1}$ term about $i$:

$$u^{n}_{i\pm 1}=u_{i}\pm \Delta xu^{n}_{ix}+\frac{\Delta x^{2}}{2} u^{n}_{ixx}\pm \frac{\Delta x^{3}}{3!} u^{n}_{ixxx}+...$$

Making the substitution into the numerical equation and rearranging gives:

$$u^{n}_{it}+cu^{n}_{ix}=-\frac{\Delta t}{2} u_{itt}-\frac{\Delta t^{2}}{3!} u_{ittt}+\frac{\Delta x^{2}}{3!} u^{n}_{ixxx}+...$$

We have recovered the original equation on the left. On the right, the spatial derivatives of even order did not survive the operation, but the odd-ordered ones, which had opposite signs, did. 

We now want the time derivatives in terms of space. We can differentiate the original equation in terms of time to obtain:

$$u_{tt}+cu_{xt}=u_{tt}+cu_{tx}=u_{tt}+c(-cu_{x})_{x}=u_{tt}-c^{2}u_{xx}=0$$

and therefore:

$$u_{tt} = c^2u_{xx}$$

Similarly, differentiating twice WRT time:

$$u_{ttt}+cu_{xtt}=u_{ttt}+cu_{ttx}=u_{ttt}+c(c^{2}u_{xx})_{x}=u_{ttt}+c^{3}u_{xxx}=0$$

and therefore:

$$u_{ttt} = -c^3u_{xxx}$$

Substituting gives:

$$u^{n}_{it}+cu^{n}_{ix}=-\frac{\Delta tc^{2}}{2} u_{ixx}+\left(\frac{\Delta t^{2}c^{3}}{3!} +\frac{\Delta x^{2}}{3!} \right)u^{n}_{ixxx}+...$$

The first term on the right will always have a negative coefficient, since both $\Delta t$ and $c^2$ will always be positive. Therefore, negative diffusion will always exist and the scheme is unconditionally unstable.

## Conduct VonNeumann stability analysis for the FTCS scheme. Find the amplification factor and show that the scheme is unconditionally unstable.